In [1]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)
        knn = self.nearest[:, 1]  
        return self.nearest

In [2]:
%%time
import numpy as np
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
min_max_scaler=preprocessing.MinMaxScaler()
#from sklearn.model_selection import KFold
#kf=KFold(n_splits=10)
def run(df):
    data=np.array(df)
    data2=np.array(df.drop('label',axis=1))
    rate=0.1
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dmax,10) #密度阈值(%10分位数)
    F1=[]
    for k in range(1,21):
        x1 = np.array([[1,1],[1,-1]])
        x2 = np.array([1,np.floor(k / 3)/ k])
        (Ps_,Pd_)=np.linalg.solve(x1,x2)
        H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
        Dn,Dn_filted= [],[]
        noise_set=pd.DataFrame(columns=df.columns)
        label_set=list(set(df.label))
        for r in label_set:    
            noise_set=pd.concat([noise_set,df[df.label==r].sample(frac=rate, replace=False)]) #选取噪声比例 
        data_cut=df[~df.index.isin(noise_set.index)]
        no=[]
        for j in range(0,len(noise_set)):
            no.append(np.random.choice(label_set))
        noise_set['label2']=no
        noise_set.reset_index(drop=True,inplace=True)
        for l in range(0,len(noise_set)):  #随机替换标签
            label_set=list(set(df.label))
            if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                label_set.remove(noise_set.loc[l,'label'])
                noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
        noise=noise_set.drop(['label'],axis=1)
        noise.rename(columns={'label2':'label'}, inplace = True)
        data= pd.concat([data_cut,noise],axis=0,ignore_index=True) #含噪声数据集
        data=np.array(data)
        kdt = KdTree()
        kdt.create(data)  
        for x in data:
            near= kdt.search(x[:-1], k+1)  # 设置临近点的个数
            density = 0
            t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
            for i in range(1,k+1):
                if x[-1] == near[i][1].data[-1]:
                    t += 1
                    hon += 1
                    dist_hon += near[i][0]
                else:
                    hen += 1
                    dist_hen += near[i][0]
                density += near[i][0]  #密度   
            DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
            DRL = (hen - hon)  #标签异同差   
            Ps = t / k
            Pd = 1 - Ps
            if (Ps==1)|(Ps==0):
                H = 0
            else:
                H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
          #  print('densit:',density)
            if (density <= a): #高密度区域
                #rint('高密度区')
                if DRL > 0:
                    Dn.append(list(x))
                   #print('高密度区噪声')
            else:
                if (H < H_): #低密度单一标签区域
                    if DRL > 0:
                        Dn.append(list(x))
                else: #低密度混合标签区域
                    DoD_ = density / k
                    if DoD >= DoD_:
                        train_hon,train_hen=[],[]
                        for j in data:
                            if j[-1]==x[-1]:
                                train_hon.append(j)
                            else:
                                train_hen.append(j)
                        train_prositive=np.array(train_hon)
                        train_negetive=np.array(train_hen)
                        kdt_hon=KdTree()
                        kdt_hen=KdTree()
                        kdt_hon.create(train_prositive)
                        kdt_hen.create(train_negetive)
                        near_hon=kdt_hon.search(x[:-1],k+1)
                        near_hen=kdt_hen.search(x[:-1],k)
                        dist_hon,dist_hen = 0,0
                        for n in near_hon:
                            dist_hon += n[0]
                        for n in near_hen:
                            dist_hen += n[0]
                        if (dist_hon ) / (dist_hen ) > 1.0:
                            Dn.append(list(x))
        noise_neighbor=[]                   
        for e in Dn:  # 计算噪声分数
            e=np.array(e)
            near2=kdt.search(e[:-1],k+1) #k近邻
            for i in range(1,k+1):
                noise_neighbor.append(list(near2[i][1].data))
        for e in Dn:
            e=np.array(e)
            near3=kdt.search(e[:-1],k+1)
            t_e=noise_neighbor.count(list(e)) #处于其它噪声近邻的次数
            confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
            neighborhood_e=0
            for i in range(1,k+1):
                t_ei=noise_neighbor.count(list(near3[i][1].data))
                confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
                if near3[i][1].data[-1]==e[-1]:
                    different_class=-1
                else:
                    different_class=1
                if list(near3[i][1].data) in Dn: 
                    isnoise=1
                else:
                    isnoise=-1
                near4=kdt.search(near3[i][1].data[:-1],k+1)
                n_ei=0
                for j in range(1,k+1):
                    if list(near4[j][1].data) in Dn:
                        n_ei += 1
                clean_ei= (k + isnoise*(n_ei-k)) / (2*k) #纯净度
                neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
            NS_e=confidence_e * neighborhood_e #噪声得分
            if NS_e > 0:
                Dn_filted.append(list(e))  

        noise_list=noise.values.tolist()
        TP = 0
        for c in Dn_filted:
            if c in noise_list:
                TP += 1
        FP = len(Dn_filted) - TP
        FN = len(noise_list) - TP
        precision= TP / (TP + FP)
        recall= TP / (TP + FN)
        f1=2 * (precision * recall) / (precision + recall) 
        F1.append(f1)
    return F1

Wall time: 3.46 s


#鸢尾花数据集

In [3]:
%%time
from sklearn.datasets import load_iris
iris = load_iris()
iris_sample = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(iris_sample)))
iris_sample['label'] = iris.target
iris_F1=run(iris_sample)

Wall time: 26.2 s


#红酒数据集

In [4]:
%%time
from sklearn.datasets import load_wine
wine= load_wine()
wine_sample = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(wine_sample)))
wine_sample['label'] = wine.target
wine_F1=run(wine_sample)

Wall time: 1min 35s


#种子数据集

In [5]:
%%time
seeds=pd.read_csv('C:/Users/liu/python works/mydata/seeds.csv',engine='python')
seeds_sample=seeds.drop(['label'],axis=1)
seeds_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(seeds_sample)))
seeds_sample['label']=seeds.label
seeds_F1=run(seeds_sample)

Wall time: 1min 13s


#glass数据集

In [6]:
%%time
glass=pd.read_csv('C:/Users/liu/python works/mydata/glass.csv',engine='python')
glass_sample=glass.drop(['label'],axis=1)
glass_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(glass_sample)))
glass_sample['label']=glass.label
glass_F1=run(glass_sample)

Wall time: 2min 15s


#蛋白质定位点数据集

In [7]:
%%time
ecoli=pd.read_csv('C:/Users/liu/python works/mydata/ecoli.csv',engine='python')
ecoli_sample=ecoli.drop(['label'],axis=1)
ecoli_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(ecoli_sample)))
ecoli_sample['label']=ecoli.label
ecoli_F1=run(ecoli_sample)

Wall time: 2min 4s


#图像分割数据集

In [8]:
%%time
image=pd.read_csv('C:/Users/liu/python works/mydata/segmentation.csv')
image_sample=image.drop(['label'],axis=1)
image_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(image_sample)))
image_sample['label']=image.label
image_F1=run(image_sample)

Wall time: 2h 54min


In [9]:
image_F1

[0.7064516129032259,
 0.8492063492063492,
 0.8231046931407943,
 0.8674242424242425,
 0.823956442831216,
 0.8312159709618875,
 0.822463768115942,
 0.8333333333333333,
 0.8214285714285715,
 0.8220640569395018,
 0.8000000000000002,
 0.8021015761821367,
 0.7972027972027971,
 0.8172043010752688,
 0.81195079086116,
 0.7747440273037542,
 0.776271186440678,
 0.7937608318890815,
 0.7796610169491526,
 0.7697478991596638]

#汽车数据集

In [10]:
%%time
statlog=pd.read_csv('C:/Users/liu/python works/mydata/statlog.csv',engine='python')
statlog_sample=statlog.drop('label',axis=1)
statlog_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(statlog_sample)))
statlog_sample['label']=statlog.label  
statlog_F1=run(statlog_sample)

Wall time: 1h 6min 11s


#yeast

In [11]:
%%time
yeast=pd.read_csv('C:/Users/liu/python works/mydata/yeast.csv',engine='python')
yeast_sample=yeast.drop(['label'],axis=1)
yeast_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(yeast_sample)))
yeast_sample['label']=yeast.label
yeast_F1=run(yeast_sample)

Wall time: 1h 6min 4s


#字母数据集

In [12]:
%%time
letter=pd.read_csv('C:/Users/liu/python works/mydata/letter.csv')
letter_sample=letter.drop(['label'],axis=1)
letter_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(letter_sample)))
letter_sample['label']=letter.label
letter_F1=run(letter_sample)

Wall time: 1h 24min 54s


#thyroid

In [13]:
%%time
thyroid=pd.read_csv('C:/Users/liu/python works/mydata/new-thyroid.csv',engine='python')
thyroid_sample=thyroid.drop(['label'],axis=1)
thyroid_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(thyroid_sample)))
thyroid_sample['label']=thyroid.label
thyroid_F1=run(thyroid_sample)

Wall time: 55.9 s


#vowel

In [14]:
%%time
vowel=pd.read_csv('C:/Users/liu/python works/mydata/vowel.csv',engine='python')
vowel_sample=vowel.drop(['label'],axis=1)
vowel_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(vowel_sample)))
vowel_sample['label']=vowel.label
vowel_F1=run(vowel_sample)

Wall time: 1h 8min 8s


#pen

In [15]:
%%time
pen_base=pd.read_csv('C:/Users/liu/python works/mydata/pen_base.csv',engine='python')
pen_sample=pen_base.drop(['label'],axis=1)
pen_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(pen_sample)))
pen_sample['label']=pen_base.label
pen_F1=run(pen_sample)

Wall time: 2h 41min 51s


In [16]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [17]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
from matplotlib import pyplot as plt
plt.figure(figsize=(8,5))
k_value=range(1,21)
plt.xticks([2,4,6,8,10,12,14,16,18,20],['2','4','6','8','10','12','14','16','18','20'])
plt.yticks([0.2,0.4,0.6,0.8,1.0],['0.2','0.4','0.6','0.8','1.0'])
plt.plot(k_value,iris_F1,'r-',label='Iris')
plt.plot(k_value,wine_F1,'g-',label='Wine')
plt.plot(k_value,seeds_F1,'b-',label='Seeds')
plt.plot(k_value,thyroid_F1,'y-',label='Thyroid')
plt.plot(k_value,statlog_F1,'c-',label='Statlog')
plt.plot(k_value,ecoli_F1,'m-',label='Ecoli')
plt.plot(k_value,glass_F1,'k-',label='Glass')
plt.plot(k_value,image_F1,'-',color='brown',label='Image')
plt.plot(k_value,letter_F1,'-',color='orange',label='Letter')
plt.plot(k_value,yeast_F1,'-',color='peru',label='Yeast')
plt.plot(k_value,pen_F1,'-',color='darkblue',label='Pen')
plt.plot(k_value,vowel_F1,'-',color='darkgray',label='Vowel')
plt.xlabel('k',fontsize=16)
plt.ylabel('F1',fontsize=16)
#plt.title('F1值曲线图')
plt.legend(bbox_to_anchor=(1.05, 0), loc=3, borderaxespad=0)
#plt.legend(loc='best',frameon=False)
plt.show()
#plt.savefig(r'./最优k值结果图/12个数据集3.png',bbox_inches='tight')

In [18]:
Fsum=(np.array(iris_F1)+np.array(wine_F1)+np.array(seeds_F1)+np.array(thyroid_F1)+np.array(statlog_F1)+np.array(ecoli_F1)+np.array(glass_F1)+np.array(image_F1)+np.array(letter_F1)+np.array(yeast_F1)+np.array(pen_F1)+np.array(vowel_F1))

In [19]:
fig2=plt.figure(figsize=(6,6))
plt.plot(k_value,Fsum,'r-')
plt.xlabel('k',fontsize=12)
plt.ylabel('Accumulated F1',fontsize=12)
plt.xticks([2,4,6,8,10,12,14,16,18,20],['2','4','6','8','10','12','14','16','18','20'])
plt.show()
#plt.savefig(r'./最优k值结果图/12个数据集fsum3.png')

In [20]:
Fsum.max()

9.106041586968077

In [21]:
Fsum

array([7.09279893, 8.81233294, 8.98439518, 9.04885588, 9.01434263,
       9.10604159, 8.794784  , 8.81196952, 8.45650839, 8.4467068 ,
       8.34472337, 8.15612902, 8.25393579, 8.14593903, 8.30801232,
       8.1873327 , 8.10008762, 8.02625849, 8.0593613 , 8.20909608])